In [2]:
from ortools.sat.python import cp_model
import math
import pandas as pd

In [32]:
hospitales_df = pd.read_csv("Hospitales.csv", sep=';')

hospitals = []
beds_per_hospital = []

for _, row in hospitales_df.iterrows():
    hospital_info = {
        "name": row["Hospitales"],
        "location": (row["Latitud"], row["Longitud"]),
    }
    hospitals.append(hospital_info)
    beds_per_hospital.append(row["Camas"])

patients_df = pd.read_csv("Patients.csv",sep=',')

patients = []

for _, row in patients_df.iterrows():
    patient_info = {
        "id": row["id"],
        "location": (row["latitude"], row["longitude"]),
        "severity": row["severity"]
    }
    patients.append(patient_info)

model = cp_model.CpModel()

x = {}
for i in range(len(hospitals)):
    for k in range(len(patients)):
        x[(i, k)] = model.NewBoolVar(f"x_{i}_{k}")

# Restricciones
# Cada paciente debe ser asignado a un solo hospital
for k in range(len(patients)):
    model.Add(sum(x[(i, k)] for i in range(len(hospitals))) == 1)

# Cada hospital solo puede tener tantos pacientes como camas disponibles
for i in range(len(hospitals)):
    model.Add(sum(x[(i, k)] for k in range(len(patients))) <= beds_per_hospital[i])

# Asignar al paciente al hospital más cercano basado en distancias euclidianas
distances = []
severity_weights = []
for i in range(len(hospitals)):
    hospital_loc = hospitals[i]["location"]
    for k in range(len(patients)):
        patient_loc = patients[k]["location"]
        severity = patients[k]["severity"]
        distance = math.sqrt(
            (hospital_loc[0] - patient_loc[0]) ** 2 + 
            (hospital_loc[1] - patient_loc[1]) ** 2
        )
        # Penalizar la asignación de pacientes con menor gravedad
        severity_weight = 5 - severity
        severity_weights.append(severity_weight * x[(i, k)])
        distances.append(distance * x[(i, k)])

model.Minimize(sum(distances) + sum(severity_weights))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or cp_model.FEASIBLE:
    print("Solucion encontrada!")
    assigned_patients = []

    for i in range(len(hospitals)):
        hospital_name = hospitals[i]["name"]
        for k in range(len(patients)):
            if solver.Value(x[(i, k)]) == 1:
                assigned_patients.append(patients[k]["id"])
                patient_id = patients[k]["id"]
                severity = patients[k]["severity"]
                print(f"El paciente con id {patient_id} de gravedad {severity} esta asignado al hospital {hospital_name}")

    patients_without_assignment = [
        patients[k]["id"] for k in range(len(patients))
        if patients[k]["id"] not in assigned_patients
    ]

    if patients_without_assignment:
        print("Pacientes sin asignacion:")
        for patient in patients_without_assignment:
            print(f"Paciente {patient}")

else:
    print("No se encontro alguna solucion")

Solucion encontrada!
El paciente con id 38.0 de gravedad 3.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 52.0 de gravedad 4.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 58.0 de gravedad 4.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 78.0 de gravedad 3.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 80.0 de gravedad 1.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 84.0 de gravedad 1.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 94.0 de gravedad 3.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 95.0 de gravedad 4.0 esta asignado al hospital Hospital de Emergencias Jose Casimiro Ulloa
El paciente con id 98.0 de gravedad 5.0 esta asignado al hospital Hospital de Emergencias Jose Casi

In [5]:
import folium
from folium.plugins import MarkerCluster

In [35]:
center_lat = hospitales_df["Latitud"].mean()
center_lon = hospitales_df["Longitud"].mean()

hospital_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

hospital_marker_cluster = MarkerCluster().add_to(hospital_map)

for _, row in hospitales_df.iterrows():
    folium.Marker(
        location=(row["Latitud"], row["Longitud"]),
        popup=row["Hospitales"],
        icon=folium.Icon(color='blue', icon='plus-sign'),
    ).add_to(hospital_marker_cluster)

patient_marker_cluster = MarkerCluster().add_to(hospital_map)

for _, row in patients_df.iterrows():
    folium.Marker(
        location=(row["latitude"], row["longitude"]),
        popup=f"Paciente ID: {row['id']}, Gravedad: {row['severity']}",
        icon=folium.Icon(color='red', icon='exclamation-sign'),
    ).add_to(patient_marker_cluster)
    
for i in range(len(hospitals)):
    hospital_loc = (hospitals[i]["location"][0], hospitals[i]["location"][1])
    hospital_name = hospitals[i]["name"]
    for k in range(len(patients)):
        if solver.Value(x[(i, k)]) == 1:
            patient_id = patients[k]["id"]
            patient_loc = (patients[k]["location"][0], patients[k]["location"][1])

            
            folium.PolyLine(
                locations=[hospital_loc, patient_loc],
                color='black',  
                weight=0.7,  
                opacity=0.5,  
                popup=f"Paciente ID: {patient_id} - Hospital: {hospital_name}"
            ).add_to(hospital_map)

hospital_map.save("Mapa_COVID.html")

hospital_map